# Quick notes in first hours of day 2

In [15]:
data = ['a','b','c'];

In [16]:
for i in 1:length(data)
    println(i,"\t", data[i])
end

1	a
2	b
3	c


In [14]:
for (i,x) in enumerate(['a','b','c'])
    println(i,"\t", x)
end

1	a
2	b
3	c


In [2]:
using InteractiveUtils: subtypes

function type_and_children(type, prefix = "", child_prefix = "")
    if isconcretetype(type)
        @assert isempty(subtypes(type))

        println(prefix, type, ": concrete")
    else
        println(prefix, type, isabstracttype(type) ? ": abstract" : ": parameterized")

        children = subtypes(type)
        for (i, c) in enumerate(children)
            if i == length(children)
                type_and_children(c, "$(child_prefix) └─╴", "$(child_prefix)    ")
            else
                type_and_children(c, "$(child_prefix) ├─╴", "$(child_prefix) │  ")
            end 
        end
    end
end;

In [3]:
subtypes(Number)

3-element Vector{Any}:
 Base.MultiplicativeInverses.MultiplicativeInverse
 Complex
 Real

In [4]:
subtypes(Real)

4-element Vector{Any}:
 AbstractFloat
 AbstractIrrational
 Integer
 Rational

In [6]:
isabstracttype(Rational)

false

In [7]:
isabstracttype(Integer)

true

In [5]:
subtypes(Any)

649-element Vector{Any}:
 AbstractArray
 AbstractChannel
 AbstractChar
 AbstractDict
 AbstractDisplay
 AbstractMatch
 AbstractPattern
 AbstractSet
 AbstractString
 Any
 Base.ANSIDelimiter
 Base.ANSIIterator
 Base.AbstractBroadcasted
 ⋮
 Tuple
 Type
 TypeVar
 UndefInitializer
 Val
 VecElement
 VersionNumber
 WeakRef
 ZMQ.Context
 ZMQ.Socket
 ZMQ.lib.zmq_msg_t
 ZMQ.lib.zmq_pollitem_t

In [17]:
type_and_children(Number)

Number: abstract
 ├─╴Base.MultiplicativeInverses.MultiplicativeInverse: abstract
 │   ├─╴Base.MultiplicativeInverses.SignedMultiplicativeInverse: parameterized
 │   └─╴Base.MultiplicativeInverses.UnsignedMultiplicativeInverse: parameterized
 ├─╴Complex: parameterized
 └─╴Real: abstract
     ├─╴AbstractFloat: abstract
     │   ├─╴BigFloat: concrete
     │   ├─╴Core.BFloat16: concrete
     │   ├─╴Float16: concrete
     │   ├─╴Float32: concrete
     │   └─╴Float64: concrete
     ├─╴AbstractIrrational: abstract
     │   └─╴Irrational: parameterized
     ├─╴Integer: abstract
     │   ├─╴Bool: concrete
     │   ├─╴Signed: abstract
     │   │   ├─╴BigInt: concrete
     │   │   ├─╴Int128: concrete
     │   │   ├─╴Int16: concrete
     │   │   ├─╴Int32: concrete
     │   │   ├─╴Int64: concrete
     │   │   └─╴Int8: concrete
     │   └─╴Unsigned: abstract
     │       ├─╴UInt128: concrete
     │       ├─╴UInt16: concrete
     │       ├─╴UInt32: concrete
     │       ├─╴UInt64: concrete
     │    

In [18]:
x = 2 + 3im;
y = 2.0 + 3im
x, y

(2 + 3im, 2.0 + 3.0im)

In [19]:
typeof(x), typeof(y)

(Complex{Int64}, ComplexF64)

In [20]:
ComplexF64

ComplexF64 (alias for Complex{Float64})

In [21]:
a = sqrt(2)

1.4142135623730951

LASSO

$$
y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \ldots \beta_p x_p
$$

LASSO helps with "model selection" in stats

## Example of RCall.jl

In [23]:
pwd()

"/Users/uqjnazar/git/Julia_ML_training"

In [24]:
using Pkg
Pkg.activate("unit1")

  Activating project at `~/git/Julia_ML_training/unit1`


In [26]:
cd("unit1"); pwd()

"/Users/uqjnazar/git/Julia_ML_training/unit1"

In [31]:
using CSV, DataFrames, RCall

data1 = CSV.read("../data/machine1.csv", header=false, DataFrame)[:,1]
data2 = CSV.read("../data/machine2.csv", header=false, DataFrame)[:,1]
data3 = CSV.read("../data/machine3.csv", header=false, DataFrame)[:,1]

function R_ANOVA(allData)
    data = vcat([ [x fill(i, length(x))] for (i, x) in
                enumerate(allData) ]...)
    df = DataFrame(data, [:Diameter, :MachNo])
    @rput df

    R"""
    df$MachNo <- as.factor(df$MachNo)
    anova <- summary(aov( Diameter ~ MachNo, data=df))
    fVal <- anova[[1]]["F value"][[1]][1]
    pVal <- anova[[1]]["Pr(>F)"][[1]][1]
    """
    println("R ANOVA f-value: ", @rget fVal)
    println("R ANOVA p-value: ", @rget pVal)
end

R_ANOVA([data1, data2, data3])

R ANOVA f-value: 10.516968568709055
R ANOVA p-value: 0.00014236168817139904


In [28]:
data1

20-element Vector{Float64}:
 53.35674558144255
 53.45887516134873
 52.282838627926125
 52.98746570643515
 51.993167774733486
 53.373333606198
 55.75410538860477
 50.279496381439365
 53.6359586914001
 53.517705831707495
 53.70044994508253
 54.15592592604583
 53.55054914606446
 52.37319589109419
 53.4900750059897
 52.939458524079605
 52.16761562743534
 50.87140009591033
 53.144919157924924
 52.09084035473537

In [30]:
? R

search:

Couldn't find R
Perhaps you meant \, ⊆, ∈, ∩, ), ℯ, ⊻, ∌, ≠, ', $, ∉, }, ", {, (, ∪, ≢, ∘ or ≉


No documentation found.

Binding `R` does not exist.


In [32]:
pwd()

"/Users/uqjnazar/git/Julia_ML_training/unit1"

In [34]:
include("work/test.jl")

hello


In [37]:
function my_factorial(n)
    @show "calling my_factorial on $n"
    if n == 0
        return 1
    else
        return n * my_factorial(n-1)
    end
end

my_factorial(10)

"calling my_factorial on $(n)" = "calling my_factorial on 10"
"calling my_factorial on $(n)" = "calling my_factorial on 9"
"calling my_factorial on $(n)" = "calling my_factorial on 8"
"calling my_factorial on $(n)" = "calling my_factorial on 7"
"calling my_factorial on $(n)" = "calling my_factorial on 6"
"calling my_factorial on $(n)" = "calling my_factorial on 5"
"calling my_factorial on $(n)" = "calling my_factorial on 4"
"calling my_factorial on $(n)" = "calling my_factorial on 3"
"calling my_factorial on $(n)" = "calling my_factorial on 2"
"calling my_factorial on $(n)" = "calling my_factorial on 1"
"calling my_factorial on $(n)" = "calling my_factorial on 0"


3628800

In [38]:
"hello" isa AbstractString

true

In [48]:
methods(split)

# 2 methods for generic function "split" from Base:
 [1] split(str::AbstractString; limit, keepempty)
     @ strings/util.jl:744
 [2] split(str::T, splitter; limit, keepempty) where T<:AbstractString
     @ strings/util.jl:737

In [49]:
x = String15("hello world")

"hello world"

In [50]:
x isa AbstractString

true

In [51]:
split(x, ' ')

2-element Vector{SubString{String15}}:
 "hello"
 "world"

In [52]:
sizeof(x)

11

In [53]:
isbits(x)

true

In [54]:
fieldnames(String15)

()